# Integrantes:
####Juan Felipe Baquero Vargas
####Lina Fernanda Rosales Castro




Se realizan los imports necesarios para la actividad

In [8]:
%matplotlib inline
import sklearn as skl
import numpy as np
import pylab as pl
import pandas as pd

##Punto 1

###A - Expresión utilizando kernerls


Dado un vecto $w$ en $X$, calcular la norma de la proyección de la imagen en un punto $x$,$\phi(x)$ en la imagen del vector $w$, $\phi(w)$:

$$P_{\phi(w)}\left(\phi(x)\right)=\frac{\langle\phi(x),\phi(y)\rangle}{\Vert\phi(w)\Vert}$$

Por la definición de kernel y la defición de la norma como producto punto: 
$\Vert x\Vert = \langle x, x\rangle^\frac{1}{2}$, se tiene que:
$$P_{\phi(w)}\left(\phi(x)\right)= \frac{ k(x,w)}{ k(w,w)^{\frac{1}{2}}}$$


### B - Varianza de las proyecciones

Calcular la varianza de las proyeciones en un espacio de un conjunto de puntos a lo largo de un vector $w$:
$$\mathrm{Var}_{\phi(w)}(x) = \frac{1}{n}\sum_{x_i\in x}\left( P_{\phi(w)}(\phi(x_i))-\mu \right)^2$$

donde

$$\mu = \frac{1}{n}\sum_{x_i\in x} P_{\phi(w)}(\phi(x_i))$$

Utilizando el resultado del literal A se tiene que:
 
 $$\mathrm{Var}_{\phi(w)}(x) = \frac{1}{n}\sum_{x_i\in x}\left( \frac{ k(x,w)}{ k(w,w)^{-\frac{1}{2}}})-\mu \right)^2$$
 
 donde 

$$\mu = \frac{1}{n k(w,w)^{\frac{1}{2}}}\sum_{x_i\in x} k(x_i,w)$$

###C - Implementacion en Python

In [132]:
#Definiciòn de kernels
def kernel1(x,w):
    return np.dot(x,w)

def kernel2(x,w):
    return np.dot(x,w)**2

def kernel3(x,w):
    return (np.dot(x,w) + 1)**5

def kernel4(x,w):
    aux = x - w
    return np.exp(-0.5 *np.dot(aux,aux))

#Varianza y proyeccion
def projection(x,w,k):
    return (k(x,w)/(k(w,w)**.5))

def variance(xs, w, k):
    n = len(xs)
    proj = np.array([projection(x,w,k) for x in xs])
    m = np.mean(proj)
    
    rest = (proj - m)**2
    
    suma = np.sum(rest)

    return (1.0/n)*suma
    
    

In [133]:
xs = np.array([[ 0,  1], 
               [-1,  3],
               [ 2,  4],
               [ 3, -1],
               [-1, -2]])

w1 = np.array([ 1, 1])

w2 = np.array([ -1, 1])

ks = [kernel1,kernel2,kernel3,kernel4]

In [135]:
wk1 = []
wk2 = []

for idx in xrange(len(ks)):
    wk1.append((variance(xs,w1,ks[idx])))


for idx in xrange(len(ks)):
    wk2.append(variance(xs,w2,ks[idx]))

wk1=np.array(wk1)
wk2=np.array(wk2)

wk1 = wk1.T
wk2= wk2.T
wi = zip(wk1,wk2)

pd.DataFrame(data=wi, columns=['$\mathbf{w}_1$', '$\mathbf{w}_2$'], index=['Kernel 1', 'Kernel 2', 'Kernel 3', 'Kernel 4'])

,$\mathbf{w}_1$,$\mathbf{w}_2$
Kernel 1,4.120000,3.720000
Kernel 2,41.340000,12.060000
Kernel 3,183362.819095,6494.993580
Kernel 4,0.056747,0.054575


##Punto 2

##Punto 3

##Punto 4